In [ ]:
import os
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
# Crear un dataset con todas las imágenes de un directorio y escalarlas al ancho y alto necesario para la arquitectura elegida

# Parámetros del notebook
path = "./test/"     # El directorio donde se leeran las imágenes
                           # El directorio a leer deberá contar con subdirectorios, uno para cada clase
                           # que se desee validar
                           # Ejemplo:
                           # ./Test
                           #     |------ Clase1
                           #     |           |------ Imagen1.jpg
                           #     |           |------ Imagen2.jpg
                           #     |           |------ Imagen3.jpg
                           #     |------ Clase2
                           #                 |------ Imagen4.jpg
                           #                 |------ Imagen5.jpg
                           #                 |------ Imagen6.jpg

augmented_path = "./augmented/" # El directorio donde almacenar las imágenes transformadas
                                # Debe existir y estar vacío

# 224x244x3 (RGB) son las dimensiones que espera VGG16
width = 224   # ancho de las imágenes necesario para la arquitectura elegida
height = 224  # alto de las imágenes necesario para la arquitectura elegida

# Función de preprocesamiento de las imágenes (propia de cada arquitectura)
preprocessing_function = keras.applications.vgg16.preprocess_input


In [ ]:
# Crear el "cargador" de imágenes
# Elegir todas las transformaciones que se deseen

datagen = ImageDataGenerator(preprocessing_function=preprocessing_function,
                             # Rota la imagen en un ángulo al azar entre (0 y 45) en sentido horario
                             rotation_range=45)

In [ ]:
# batch_size tiene un número alto para asegurarse que todas las imágenes entren en el mismo y único lote

generator = datagen.flow_from_directory(path, target_size=(height, width),
                                        save_to_dir=augmented_path, batch_size=9999)

# Obtengo el único lote con todas las imágenes
batch_images = next(generator)
n_images = len(batch_images[0])

# Lectura de los archivos con las imágenes transformadas
modified_images = os.listdir(augmented_path)
for i in range(len(modified_images)):
  fn = modified_images[i]
  fn = fn[1:]
  uc = fn.find("_")
  if uc == 1:
    fn = "0" + fn
  modified_images[i] = fn
modified_images.sort()
for i in range(len(modified_images)):
  fn = modified_images[i]
  if fn[0] == "0":
    fn = fn[1:]
  modified_images[i] = "_" + fn

# Obtener el listado de los archivos leidos
file_names = generator.filenames

# Cargar el nombre de las clases de ImageNet
content = open("ImageNet.classes").readlines()
class_names = [line[9:].strip() for line in content]

In [ ]:
# Cargar el modelo preentrenado, puede usar cualquier arquitectura vista en clase, otra provista por keras o alguna otra creada por usted

model = keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
model.summary()
keras.utils.plot_model(model)

In [ ]:
# Validación del conjunto de testeo
# Usar el modelo cargado para hacer la predicción

for i in range(n_images):
  image = batch_images[0][i]
  file_name = file_names[i]

  x = np.expand_dims(image, axis=0)
  results = model.predict(x, verbose=False)
  pred = np.argmax(results, axis=1)
  pred_class = class_names[pred[0]]

  print(f'Class: {pred_class}\tProb. of class = {results[0][pred[0]]}')

  image = load_img(path + file_names[i],color_mode='rgb', target_size=(224, 224))
  display(image)

  image = load_img(augmented_path + modified_images[i],color_mode='rgb', target_size=(224, 224))
  display(image)

  print("-----------------------------------------------------------------")
